# Chess with minimax

In [ ]:
import chess
import pygame

pygame.init()
width = 400
height = 400
screen = pygame.display.set_mode([width, height])
pygame.display.set_caption("Simple Pygame Chess!")
speed = pygame.time.Clock()
fps = 60 #frames per second

In [ ]:
images_dict = {}
def load_images():

    piece_images = {
        "P": "White Pawn",
        "N": "White Knight",
        "B": "White Bishop",
        "R": "White Rook",
        "Q": "White Queen",
        "K": "White King",
        "p": "Black Pawn",
        "n": "Black Knight",
        "b": "Black Bishop",
        "r": "Black Rook",
        "q": "Black Queen",
        "k": "Black King"
    }

    for piece in piece_images:
        passage = "C:\\Users\\Sorena\\Desktop\\chess icons\\" + piece_images[piece] + ".png"
        images_dict[piece_images[piece]] = pygame.image.load(passage)    #image_dict["white knight"]
        images_dict[piece_images[piece]] = pygame.transform.scale(images_dict[piece_images[piece]], (50, 50))

In [ ]:
board = chess.Board()

## Drawing board based on chess library

In [ ]:
def draw_board(screen): 
    for row in range(8):
        for coloumn in range(8):        #0,0 must be light
            if (row + coloumn) % 2 == 0:
                color = pygame.Color("white")
            else:
                color = pygame.Color("dark green")
            
            size = 400//8
            pygame.draw.rect(screen, color, pygame.Rect(coloumn*size, row*size, size, size))

In [ ]:
def put_pieces():
    for square in chess.SQUARES:
        piece = board.piece_at(square)
        
        if piece:
            if piece.color == chess.WHITE:
                color = "White"
            elif piece.color == chess.BLACK:
                color = "Black"
            
            if str(piece.symbol()).lower() == "r":
                name = "Rook"
            if str(piece.symbol()).lower() == "n":
                name = "Knight"
            if str(piece.symbol()).lower() == "b":
                name = "Bishop"
            if str(piece.symbol()).lower() == "q":
                name = "Queen"
            if str(piece.symbol()).lower() == "k":
                name = "King"
            if str(piece.symbol()).lower() == "p":
                name = "Pawn"
                
            col = chess.square_file(square)
            row = 7 - chess.square_rank(square)     # turning the board
            screen.blit(images_dict[color + " " + name], (col * 50, row * 50))
            # bottom left is 0 = white rook, top right is 63 = black rook

In [ ]:
piece_values = {
        chess.PAWN: 1,
        chess.KNIGHT: 3,
        chess.BISHOP: 3,
        chess.ROOK: 5,
        chess.QUEEN: 9,
        chess.KING: 100
    }


In [ ]:
white_score = 0
black_score = 0
def get_score(piece2):
    global white_score, black_score
    # piece2 = pieces_on_board.get(str(go_square))  # check if second square is empty
    
    if piece2:
        value = piece_values.get(piece2.piece_type, 0)
        if piece2.color == chess.WHITE:
            black_score += value
        else:
            white_score += value
    print("White score:", white_score)
    print("Black score:", black_score)

In [ ]:
def move_piece(move):
    captured = board.piece_at(move.to_square)
    get_score(captured)
    board.push(move)

In [ ]:
def pos_to_row(x, y):
        c = x // (400//8)
        r = y // (400//8)
        square = chess.square(c, 7-r)
        return square

In [ ]:
def evaluate_board(board):  # for minmax we evaluate ((white_score - black_score)),
                            # so the white player is maximizing player and black is minimizing player
    score = 0
    for square in chess.SQUARES:
        piece = board.piece_at(square)
        if piece is not None:
            value = piece_values[piece.piece_type]
            if piece.color == chess.WHITE:
                score += value
            else:
                score -= value
    return score

In [ ]:
def minmax(board, depth, maximizing):
    # i = 1
    if depth == 0 or board.is_game_over():
        return evaluate_board(board)
    
    # white agent
    if maximizing:
        max_score = -1000
        for move in board.legal_moves:
            board.push(move)
            score = minmax(board, depth-1, False)
            board.pop()         # undo the move
            if score > max_score:
                max_score = score
                # print("max_score :", max_score)
        return max_score

    # black agent
    else:
        min_score = 1000
        for move in board.legal_moves:
            board.push(move)
            score = minmax(board, depth-1, True)
            # print("depth is :", depth)
            # print("move :",i, move)
            board.pop()         # undo the move
            # i+=1
            if score < min_score:
                min_score = score
                print("min_score :", min_score)
        return min_score

In [ ]:
def find_best_move(depth = 2):       # for black agent only 
    min_score = 1000
    best_move = None
    for move in board.legal_moves:
        board.push(move)
        score = minmax(board, depth - 1, True)
        board.pop()
        if score < min_score:
            min_score = score
            best_move = move
    print("next_move :", best_move)
    return best_move   # returns the best move based on the score of each move in the specifeid depth

In [ ]:
def game():
    running = True
    selected_square = None

    while running:
        load_images()
        draw_board(screen)
        put_pieces()
        speed.tick(fps)
        pygame.display.flip()

        # for black's turn, agent plays
        if not board.turn:
            best = find_best_move()
            if best:
                move_piece(best)

        if board.is_checkmate():
            print("Checkmate! Game Over!")
            running = False
        elif board.is_stalemate():
            print("Stalemate! Game Draw!")
            running = False
        elif board.is_check():
            print("Check!")

        #event handling
        for event in pygame.event.get():
            if event.type == pygame.QUIT:           # for exiting the loop and game
                running = False
            
            elif event.type == pygame.MOUSEBUTTONDOWN and board.turn:    # for noticing clicks
                x, y = pygame.mouse.get_pos()
                go_square = pos_to_row(x, y)
                # print(x, y)
                # go_square = go_square[0]*8 + go_square[1]    # bottom left is 0 , top right is 63
                # print(square)
                # piece = board.piece_at(go_square)     # check if there's a piece on the selected square
                                                        # returns None or chess.Piece(object)

                if selected_square is None:         # first click
                    piece = board.piece_at(go_square)
                    if piece and piece.color == chess.WHITE:              # if click is on icons
                        # print(piece)
                        selected_square = go_square                            
                        # print(selected_square)

                else:                               # second click, must move if possible
                    move = chess.Move(selected_square, go_square)  # from - to

                    if move in board.legal_moves:
                        move_piece(move)

                        if board.is_checkmate():
                            print("Checkmate! Game Over!")
                            running = False

                        elif board.is_stalemate():
                            print("Stalemate! Game Draw!")
                            running = False

                        elif board.is_check():
                            print("Check!")

                    else:
                        print("illigal move")
                    
                    selected_square = None          # for next move
        
game()